# Chapter 8: I've Got the Power: Introduction to Power Analysis

This notebook is a companion to Chapter 8 of The Hardware Hacking Handbook by Jasper van Woudenberg and Colin O'Flynn. The headings in this notebook follow the headings in the book.

© 2021. This work is licensed under a [CC BY-SA 4.0 license](https://creativecommons.org/licenses/by-sa/4.0/). 

## SPA on ECDSA

### Goal and notation, finding the leaky operation

In [ ]:
# Some code from Simple ECDSA sage notebook (greg@xiph.org)
# https://github.com/TheBlueMatt/bitcoinninja/blob/master/secp256k1.ecdsa.sage

import numpy as np
from fastecdsa import keys, point, curve
import random
from gmpy2 import invert, t_mod
from bokeh.io import output_notebook, export_svgs
from bokeh.plotting import figure, show
from bokeh.models import Label
from itertools import groupby

np.random.seed(1337) # Make sure we generate same results every time
random.seed(1337)

In [ ]:
# Normal distribution for add/double operations (mean,stddev). 
# You can tune this to create more or less overlap in the distributions
TIMING=(10,2.1)

# Implement textbook elliptic curve scalar multiplication
# But leak timing info, like in an SPA trace
def leaky_scalar_mul(secret, P):
    Q = point.Point.IDENTITY_ELEMENT
    time = 0
    timeleak = [time] # Leak time
    
    # Loop over secret bits
    while secret > 0:
        keybit = secret & 1
        if keybit: # Bit set to 1? Point addition
            Q = Q + P
            time += np.random.normal(TIMING[0], TIMING[1])
            
        P = 2*P # Always point doubling
        secret = secret >> 1

        time += np.random.normal(TIMING[0], TIMING[1])
        timeleak.append(time) # Leak time

    return timeleak, Q

# Do a leaky ECDSA sign over curv with message e and private key d, optionally given k
def ecdsa_sign_leaky(curv, e, d, k=None):
    if not k:
        k = random.getrandbits(d.bit_length()) # get nonce
    
    timeleak, R = leaky_scalar_mul(k, curv.G)  
    
    # ECDSA signature
    r = R.x
    s = t_mod(invert(k, curv.q) * (e + r * d), curv.q) # s = (1/k)*(e+r*d) mod q

    return timeleak, r, s, k

# ECDSA signature verification
def ecdsa_verify(curv, r, s, e, pd):
    s_inv = int(invert(s, curv.q))        # s^-1 mod q
    r_calc = (s_inv * e * curv.G + r * s_inv * pd).x  # s^-1 * e * G + r * s^-1 * pd 
    verifies = r == r_calc # Should be True!
    return verifies

# Generate a random message and pub/private keypair for specified curv
def gen_msg_and_key(curv):
    d, pd = keys.gen_keypair(curv) # random secret and our pubkey
    e = random.getrandbits(curv.p.bit_length()) # random message

    return e, d, pd

In [ ]:
# Perform leaky ECDSA signature and check signature. This is just for functional correctness.
curv = curve.secp256k1 
e, d, pd = gen_msg_and_key(curv)
timeleak, r, s, k = ecdsa_sign_leaky(curv, e, d)
verifies = ecdsa_verify(curv, r, s, e, pd)

print("Private key d:", bin(d))
print("Public point pd: ", pd)
print("Message e:", bin(e)) 
print("Secret nonce k:", bin(k))
print("Signature verifies (should be True):", verifies)

### Simulating SPA traces of a leaky ECDSA

In [ ]:
SAMPLES_PER_LOOP=10 # This is a 'sampling rate'. Higher means more precise time measurement.
SCOPE_NOISE=0.05    # This is the amount of 'acquisition noise' to add to the measurement. 

# Take the time leakages, and create a simulated trace
def timeleak_to_trace(timeleak, samples_per_loop, scope_noise):
    points = len(timeleak)*SAMPLES_PER_LOOP  # Trace length samples
    timeleak0 = np.asarray(timeleak) - timeleak[0] # Zero-base time
    timeidx = (timeleak0 / timeleak0[-1] * (points-1)).astype(int) # Convert to int indices
    trace = np.ones(points) # Set all to power value 1
    trace[timeidx] = 0 # Except when switch between operations

    # Add measurement noise
    trace += np.random.normal(0, scope_noise, len(trace))
    
    return trace, timeleak0

In [ ]:
# Plot a trace 
TRIM = 200  # Initial zoom in samples

trace, timeleak0 = timeleak_to_trace(timeleak, SAMPLES_PER_LOOP, SCOPE_NOISE)

# Plot simulated trace
output_notebook()
p = figure(x_range=(0,TRIM), x_axis_label='"Time"', y_axis_label='"Volt"', title="ECDSA sign trace with actual nonce bits k")

p.line(range(len(trace)), trace, line_color="blue")

# Plot known bits
op_idx = timeleak0 / timeleak0[-1] * (len(trace)-1)+0.5
for bit in range(len(op_idx)):
    p.add_layout(Label(x=op_idx[bit], y=0.8, text=str(k>>bit&1), text_color="red"))

show(p)
p.output_backend = "svg"
export_svgs(p, filename="fig/8_8.svg")

### Measuring scalar multiplication loop duration 

In [ ]:
# This is where SPA magic happens. We turn a sampled trace into a list of durations between peaks
def trace_to_difftime(trace):
    threshold = (min(trace) + max(trace)) / 2 # Threshold on the y axis
    trace_bin = (trace > threshold)*1 # Turn trace into a binary representation over/under threshold
    
    # This takes the binary trace and turns it into run-lengths, as follows:
    # [0, 1, 1, 0, 1, 0, 1, 1, 1] => 
    # groupby() => 
    # (0,[0]), (1,[1, 1]), (0,[0]), (1,[1]), (0,[0]), (1,[1, 1, 1]) => 
    # filter for val==1 => 
    # [1, 1], [1], [1, 1, 1] => 
    # sum(1 for _ in group) => (technical note: groups aren't lists but iterators, therefore we use sum)
    # 2, 1, 3
    trace_rle = np.asarray([sum(1 for _ in group) for val,group in groupby(trace_bin) if val==1])
    
    return trace_rle, trace_bin, threshold

In [ ]:
# Let's plot the binary trace, threshold, and distances between peaks
trace_rle, trace_bin, threshold = trace_to_difftime(trace)

# Init figure
output_notebook()
p = figure(x_range=(0,TRIM), x_axis_label='"Time"', y_axis_label='"Binary"', title="ECDSA sign binary trace with peak distances at given threshold")

# Binary trace
p.line(range(len(trace)), trace_bin, line_color="blue")

# Threshold line
p.line([0, len(trace)], [threshold,threshold], line_width=4, color='black', line_dash='dashed')

# Plot distances
for idx in range(len(trace_rle)):
    p.add_layout(Label(x=op_idx[idx], y=0.8, text=str(trace_rle[idx]), text_color="green"))

show(p)
p.output_backend = "svg"
export_svgs(p, filename="fig/8_9.svg")

### From durations to bits

In [ ]:
# This performs the Simple Power Analysis by turning given traces into peak distances, 
# and in turn turning these distances into bits of guessed nonce k.
def simple_power_analysis(trace):
    # Turn trace into 'time differences between peaks'
    difftime, _, _ = trace_to_difftime(trace)
    
    # Determining the cutoff is quite critical. 
    # We take the midpoint between 25th and 75th percentile, which is less sensitive to outliers 
    # than taking the average. For some distributions, average works better. YMMV.
    cutoff = (np.percentile(difftime, 75) + np.percentile(difftime, 25)) / 2
    
    bitguess_rev = (difftime > cutoff)*1 # Guess bits based on cutoff
    bitguess = bitguess_rev[::-1] # Reverse array

    # Create k we guess without bruteforce
    guessed_k_base = int("".join([str(x) for x in bitguess.tolist()]),2) # Convert to int, optimized for LOC
    
    # Find bits closest to the cutoff
    cutoff_dist = np.abs(difftime - cutoff) # Calculate distance to cutoff for each bit
    closest = np.argsort(cutoff_dist) # Sort by distance, get index 
    
    return closest, guessed_k_base, difftime, cutoff

In [ ]:
# Create a power-like trace
_, _, difftime, cutoff = simple_power_analysis(trace)

In [ ]:
# Let's have a look at the two distributions of time for double only or double+add
BINS=7 # Number of bins per histogram

bitactual = np.asarray([int(k & (1<<x) > 0) for x in range(0,len(difftime))]) # Turn k into bit vector

# Plot two histograms and cutoff
output_notebook()
p = figure(x_axis_label="Duration", y_axis_label="Count", title="Distributions for Double vs Double+Add, shown with cutoff")

# Calc histograms for bits 0 and 1
hist_0,edges_0 = np.histogram(difftime[bitactual==0], bins=BINS)
hist_1,edges_1 = np.histogram(difftime[bitactual==1], bins=BINS)

# Plot histograms
p.quad(top=hist_0,bottom=0,left=edges_0[:-1],right=edges_0[1:],fill_color='red',line_color='black', fill_alpha=0.5)
p.quad(top=hist_1,bottom=0,left=edges_1[:-1],right=edges_1[1:],fill_color='blue',line_color='black', fill_alpha=0.5)

# Add cutoff as vertical line
p.line([cutoff, cutoff], [0,np.max([hist_0, hist_1])], line_width=4, color='black', line_dash='dashed')

show(p)
p.output_backend = "svg"
export_svgs(p, filename="fig/8_10.svg")

### Bruteforcing our way out

In [ ]:
# Since we have some overlapping distributions, we'll have bit errors we'll need to bruteforce
# This function does a bruteforce of the bits closest to the decision boundary
def bruteforce(bits, guessed_k_base, closest, curv, r, s, e, pd, d):
    print("Bruteforcing bits:", closest[:bits])
    
    for i in range(0, 2**bits):
        guessed_k = guessed_k_base
        
        # Flip bits in guessed_k according to our bruteforce counter i
        for x in range(bits):
            bitset = (i>>x) & 1         # 1 or 0
            bitval = 1<<int(closest[x]) # Get index to flip
            guessed_k = guessed_k ^ (bitset * bitval) # Flip correct bit iff bitset==1
           
        # Computed d from guessed k and signature
        computed_d = t_mod((s*guessed_k-e) * invert(r, curv.q), curv.q)
        
        if d:
            # If we have knowledge of d, just directly compare (fast)
            done = computed_d == d
        else:
            # Validate private key d by signing message e and verifying we get the same signature
            _, r2, s2, _ = ecdsa_sign_leaky(curv, e, computed_d, guessed_k)
            done = (r==r2 and s==s2)
        
        if done:
            return computed_d
        
    return None


In [ ]:
MAX_ATTEMPTS = 200  # Number of times to perform a leaky signing
BRUTEFORCE_BITS = 8 # Number of bits to bruteforce
GOD_MODE = True     # True => speed things up by using knowledge of the key, False => what needs to be done if you don't know the key

for attempt in range(MAX_ATTEMPTS):    
    print("\n\nAttempt", attempt)

    # Perform leaky ECDSA signature
    e, d, pd = gen_msg_and_key(curv)
    timeleak, r, s, k = ecdsa_sign_leaky(curv, e, d)
    verifies = ecdsa_verify(curv, r, s, e, pd)
    assert(verifies)

    # Create trace and do SPA
    trace, _ = timeleak_to_trace(timeleak, SAMPLES_PER_LOOP, SCOPE_NOISE)
    closest, guessed_k_base, difftime, cutoff = simple_power_analysis(trace)
    print("Guessed k:", bin(guessed_k_base))

    if GOD_MODE:
        print("Actual k: ", bin(k))
        
        # Check number of bit errors, skip bruteforce if there are too many
        biterrors = bin(guessed_k_base ^ k).count('1')
        print("Bit errors:", biterrors)
        if biterrors > BRUTEFORCE_BITS:
            print("We won't bruteforce our way out of is. Let's just try a new one.")
            continue
        
        # Provide actual private key d to bruteforce
        key = bruteforce(BRUTEFORCE_BITS, guessed_k_base, closest, curv, r, s, e, pd, d)
    else:
        # Don't know private key d
        key = bruteforce(BRUTEFORCE_BITS, guessed_k_base, closest, curv, r, s, e, pd, None) 
    
    # Done?
    if key:
        print("Yeash! Key found:", bin(key))
        break
    else:
        print("No key for you.")